## 数据追踪理论

* 上一章我们看到了汉明重量和功率的关系，现在我们考虑一些随机数据和噪音的情况。

In [ ]:
#Capture Traces
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()

traces = []

for i in tnrange(num_traces, desc='Capturing traces'):
    key, text = ktp.next()
    
    #Currently ALL bits are random. Let's extend bit 0 to a full byte to give us a random 0xFF or 0x00
    if text[0] & 0x01:
        text[0] = 0xFF
    else:
        text[0] = 0x00
    
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    traces.append(trace)

![](img/bokeh_plot_2.png)

## 数据分析

* 我们要做这些事情

    1. 对轨迹进行分组
    2. 求每组的平均值
    3. 查看差分的区别

    ![](img/bokeh_plot_3.png)

* （Xenny）显然我们可以看到一个尖峰数据，这说明了HW值的一个影响幅度。